## Example of usage of the Python interface to the circuit simulator PAN

In [ ]:
import pypan.ui as pan
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The purpose of this notebook is to show some of PAN's basic functionalities that can be accessed by means of the Python interface.

We will use a netlist that implements the Van der Pol oscillator, whose equations are the following:

$$
\begin{equation*}
\left\{
\begin{array}{rcl}
\dot{x} & = & \epsilon\ (1 - x^2)\ x - y \\
\dot{y} & = & x \ ,
\end{array}
\right.
\end{equation*}
$$

where $x$ represents the voltage across the capacitor, $y$ is the current in the inductor and the parameter $\epsilon$ is a positive constant that determines the frequency of the oscillator. If $\epsilon \ll 1$, the oscillating frequency is equal to $2\pi$, whereas if $\epsilon$ is larger, the frequency is a function of $\epsilon$ itself.

We will start with $\epsilon=0.001$ and then increase it to 5.

The following netlist defines the Van der Pol circuit in PAN.

In [ ]:
netlist = """\
ground electrical gnd

parameter EPS=1m

; This dummy alter is necessary to be able to change the value of the parameter from Python
Alrt alter param="EPS" rt=yes

; The circuit

p1    x   gnd  x    vccs func=-EPS*v(x) + EPS*v(x)^3
c1    x   gnd  capacitor c=1 icon=2m
l1    x   gnd  inductor  l=1
"""

Write the above netlist to file.

In [ ]:
netlist_file = 'vanderpol.pan'
with open(netlist_file, 'w') as fid:
    fid.write(netlist)

### Load the netlist with the description of the circuit
No analysis is performed at this point, since none is defined in the netlist.

In [ ]:
ok,libs = pan.load_netlist(netlist_file)
if not ok:
    print('load_netlist failed.')

### Run a first (short) transient analysis

In [ ]:
tstop = 1000
mem_vars = ['time', 'x', 'l1:i']
tran = pan.tran('Tr', tstop, mem_vars, libs, uic=1, vreltol=1e-6, vabstol=1e-9)

### Run a longer envelope analysis
The dynamics of the system are very slow, hence it is convenient to use an envelope analysis to speed up the integration of the system.

In [ ]:
tstop = 50e3
T = 2*np.pi
env = pan.envelope('En', tstop, T, mem_vars, autonomous='yes', restart='no')

### Run a shooting analysis
We can confirm that the envelope integration has reached the steady-state by comparing the last cycle obtained with the envelope with the one found by means of the shooting procedure.

In [ ]:
shoot_1 = pan.shooting('Sh1', T, mem_vars, autonomous='yes', restart='no')

### Alter the value of a parameter
We increase the value of $\epsilon$ from 0.001 to 5.

In [ ]:
pan.alter('Al', 'EPS', 5, invalidate='no', annotate=4);

### Run a second shooting analysis
We want to see how much the shape of the limit cycle has changed with the new value of $\epsilon$.

In [ ]:
shoot_2 = pan.shooting('Sh2', T, mem_vars, autonomous='yes', restart='no')

### Plot the results

In [ ]:
fig,ax = plt.subplots(2, 2, figsize=(12,8))
for i in range(1):
    ax[0,i].plot(tran[0], tran[i+1], 'k', linewidth=1, label='Tran')
    ax[0,i].plot(env[0], env[i+1], 'r', linewidth=1, label='Envelope')
    ax[0,i].plot(shoot_1[0], shoot_1[i+1], 'g', linewidth=1, label='Shooting-1')
    ax[0,i].plot(shoot_2[0], shoot_2[i+1], 'm', linewidth=1, label='Shooting-2')
    ax[0,i].set_xlabel('Time [s]')

ax[0,1].plot(shoot_1[1], shoot_1[2], 'g', linewidth=2, label='Shooting-1')
ax[0,1].plot(shoot_2[1], shoot_2[2], 'm', linewidth=2, label='Shooting-2')
    
ax[0,0].set_ylabel(r'$v_C$ [V]')
ax[0,1].set_xlabel(r'$v_C$ [V]')
ax[0,1].set_ylabel(r'$i_L$ [A]')

ax[0,0].legend(loc='best')

idx, = np.where(tran[0] > 950)
ax[1,0].plot(tran[0][idx], tran[1][idx], 'k', linewidth=1)
idx, = np.where(env[0] < 1100)
ax[1,0].plot(env[0][idx], env[1][idx], 'r', linewidth=1)
ax[1,0].set_xlabel('Time [s]')
ax[1,0].set_ylabel(r'$v_C$ [V]')

idx, = np.where(env[0] > tstop-1000)
ax[1,1].plot(env[0][idx], env[1][idx], 'r', linewidth=2)
ax[1,1].plot(shoot_1[0], shoot_1[1], 'g', linewidth=2)
ax[1,1].plot(shoot_2[0], shoot_2[1], 'm', linewidth=2)
ax[1,1].set_xlabel('Time [s]')
ax[1,1].set_ylabel(r'$v_C$ [V]')
ax[1,1].set_xlim([tstop - 5*np.pi, shoot_2[0][-1]])

fig.tight_layout()